<a href="https://colab.research.google.com/github/scandey/gammakde/blob/main/KDEComptonanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(rc={'figure.figsize':(12,8)})
sns.set_context("poster")
!pip install KDEpy

In [ ]:
with open('CesiumSheetTest.inc2.id1.tra','r') as fh:
     all_lines = fh.readlines()

energies = []
for line in all_lines:
    if line[:2] != 'PE':
        continue
    else:
        [PE,val] = line.split()
        energies.append(float(val))
print(energies)

In [ ]:
from KDEpy import FFTKDE
sns.set_style("ticks")

cleandata = np.array(energies)
data = cleandata[cleandata > 100 ] # remove everything under 100
points = len(data)
kde = FFTKDE(bw=10)
x, y = kde.fit(data)(points)  # Two-step proceudure to get bw
fig, ax1 = plt.subplots()
color = 'R'
ax1.set_xlabel('Energy (keV)')
ax1.plot(x, y, color='firebrick', label='KDE')
ax1.set_ylabel('Normalized KDE', color='firebrick')
ax2 = ax1.twinx()  
ax2.hist(data, alpha=0.3, color='navy',label='Histogram')
ax2.set_ylabel('Count',color='navy')
plt.title('MEGAlib simulation KDE and histogram')
red_patch = mpatches.Patch(color='firebrick', label='KDE curve')
blue_patch = mpatches.Patch(color='mediumpurple', label='Histogram')
plt.legend(handles=[red_patch,blue_patch])
plt.show()

In [ ]:
from scipy import signal
from KDEpy import FFTKDE
import matplotlib.patches as mpatches
sns.set_style("ticks")

cleandata = np.array(energies)
data = cleandata[cleandata > 100 ] # remove everything under 100 to prevent boundary condition issues
points = len(data)
kde = FFTKDE(bw=10)
x, y = kde.fit(data)(points)
fig, ax1 = plt.subplots()
ax1.set_xlabel('Energy (keV)')
ax1.plot(x, y, label='FFTKDE', color='firebrick')
ax1.set_ylabel('Normalized KDE', color='firebrick')

ax2 = ax1.twinx() 
dx = x[1]-x[0]
dydx = np.gradient(y, dx)
ax2.plot(x,dydx, color='navy')
ax2.set_ylabel('Gradient of KDE',color='navy')

red_patch = mpatches.Patch(color='firebrick', label='KDE curve')
blue_patch = mpatches.Patch(color='navy', label='Gradient of KDE')
plt.legend(handles=[red_patch,blue_patch])

peaks, _ = signal.find_peaks(-dydx, height = 0.5e-5, distance=100000)
plt.vlines(x=x[peaks], ymin = -2.5e-5, ymax = 0, color='purple')
plt.text(400, -2e-5, '506.5 keV', horizontalalignment='center', verticalalignment='center',color='purple')
plt.title('MEGAlib Simulation KDE and Gradient')
print(x[peaks])